In [1]:
import mysql.connector as mdb
import ujson as js
import pandas as pd
import numpy as np

In [2]:
with open('./../settings/maria_login.json') as file:
    conn = mdb.connect(**js.load(file))

In [3]:
mkt_grps = np.array([
    512, 514, 515, 516, 517, 518, 519, 521, 522, 523, 525, 526, 527, 528, 529, 530, ## Standard Ores
    2396, 2397, 2398, 2400, 2401, ## Moon Ores
    1855, ## Ice Ores
])

In [21]:
sql = """\
SELECT Types.type_id,
	MktGrp.path,
	Types.type_name,
    Types.portion_size,
    Types.volume
FROM Types
JOIN TypeMarketGroups AS MktGrp
	ON Types.market_group_id = MktGrp.market_group_id
WHERE Types.market_group_id IN ({mkt_grp_ids})
;""".format(mkt_grp_ids=','.join(mkt_grps.astype(str)))

ore_data = pd.read_sql(sql, conn, index_col='type_id')
ore_data['compressed'] = ore_data['type_name'].str.contains('Compressed')
ore_data['parent'] = None
ore_data.loc[ore_data['compressed'], 'parent'] = ore_data.loc[ore_data['compressed'], 'type_name'].str.replace('Compressed ', '')
ore_data['path'] = ore_data['path'].str.replace('Manufacture & Research.Materials.Raw Materials.', '')

In [22]:
ore_data

,path,type_name,portion_size,volume,compressed,parent
type_id,,,,,,
22,Standard Ores.Arkonor,Arkonor,100,16.0,False,None
17425,Standard Ores.Arkonor,Crimson Arkonor,100,16.0,False,None
17426,Standard Ores.Arkonor,Prime Arkonor,100,16.0,False,None
28367,Standard Ores.Arkonor,Compressed Arkonor,1,8.8,True,Arkonor
28385,Standard Ores.Arkonor,Compressed Crimson Arkonor,1,8.8,True,Crimson Arkonor
...,...,...,...,...,...,...
46315,Moon Ores.Exceptional Moon Ores,Shining Monazite,100,10.0,False,None
46316,Moon Ores.Exceptional Moon Ores,Bountiful Loparite,100,10.0,False,None
46317,Moon Ores.Exceptional Moon Ores,Shining Loparite,100,10.0,False,None
